In [1]:
import pandas as pd

content = pd.read_parquet("data/content-sample.parquet")
content.head()

,id,url,provider,metadata,content,statistics,load_id,load_label,load_ts,load_date
0,45445160-a2dd-437e-b7ce-8a0d32da4e23,https://api.github.com/repos/didi/logbook,github,"{""created_at"":""2022-01-12T08:29:01Z"",""descript...","{""readme"":""# logbook 简介\nlogbook 是一款面向 ToB 业务的...","{""forks"":9,""issues"":0,""stars"":103}",203ac5aa-fb5c-4f4b-ad54-8ce27ddd0622,None,2023-06-16 00:38:04,2023-06-16
1,e625534c-4792-4a64-964e-3559349eb66c,https://api.github.com/repos/scofield7419/THOR...,github,"{""created_at"":""2023-05-18T13:06:06Z"",""descript...","{""readme"":""## ����‍♀️⚡⚒️ THOR: Three-hop Reaso...","{""forks"":1,""issues"":0,""stars"":104}",203ac5aa-fb5c-4f4b-ad54-8ce27ddd0622,None,2023-06-16 00:38:04,2023-06-16
2,46878b47-b242-4c8d-9300-855fbb225947,https://api.github.com/repos/scofield7419/LAGC...,github,"{""created_at"":""2023-06-11T12:57:26Z"",""descript...","{""readme"":""# LAGCN SRL Pointer\nThis repositor...","{""forks"":0,""issues"":0,""stars"":50}",203ac5aa-fb5c-4f4b-ad54-8ce27ddd0622,None,2023-06-16 00:38:04,2023-06-16
3,d7fc273a-ce2c-42b7-9bb3-c6ccb1ca3413,https://api.github.com/repos/scofield7419/DiaR...,github,"{""created_at"":""2022-04-24T08:25:20Z"",""descript...","{""readme"":""**Python implementation of IJCAI 20...","{""forks"":0,""issues"":0,""stars"":53}",203ac5aa-fb5c-4f4b-ad54-8ce27ddd0622,None,2023-06-16 00:38:04,2023-06-16
4,896542c4-ad6b-46e0-9b9e-aba6cfa13838,https://api.github.com/repos/kdoctor-io/kdoctor,github,"{""created_at"":""2023-06-05T03:59:18Z"",""descript...","{""readme"":""# kdoctor\n[![Auto Release Version]...","{""forks"":10,""issues"":6,""stars"":62}",203ac5aa-fb5c-4f4b-ad54-8ce27ddd0622,None,2023-06-16 00:38:04,2023-06-16


In [2]:
content.shape

(7566, 10)